In [2]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
video_folder_path = ".\BetaMove\\videos"
rawImage_folder_path = ".\BetaMove\\rawImages"

In [4]:
print(os.curdir)
#print(os.getcwd()+ "\.venv\BetaMove\\videos")
os.getcwd()+ "\.venv\BetaMove\\videos"

.


'C:\\Users\\Connor\\Documents\\1School\\AiAndMl\\Term2\\INFO-6156Capstone\\.venv\\.venv\\BetaMove\\videos'

In [ ]:
# image = cv2.imread(rawImage_folder_path + "video01.mp4_frame_0000.jpg")

In [7]:
def extract_frames_every_second(video_path, filename, output_folder, interval_seconds=1.0):
    """
    Extracts one frame per second from a video and saves them as images.

    Args:
        video_path (str): The path to the input video file.
        filename (str): The input video filename.
        output_folder (str): The folder where extracted images will be saved.
        interval_seconds (float): take a picture every # frames, default is 1 per second
    """

    #Check if output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    full_file_path = video_path + "\\" + filename
    # open the video with cv2
    cap = cv2.VideoCapture(full_file_path)

    #check if the video opend successifully
    if not cap.isOpened():
        print(f"Error: Could not open video file {full_file_path}")
        return
    
    # get the num fps (24, 30, ect)
    fps = cap.get(cv2.CAP_PROP_FPS)
    #frame_interval = int(round(fps * interval_seconds))  # frames to skip for desired seconds
    frame_interval = int((fps * interval_seconds))  # frames to skip for desired seconds
    # print(frame_interval)
    if frame_interval <= 0:
        frame_interval = 1  # Ensure at least 1 frame step

    frame_count = 0
    saved_frames_count = 0

    while True:
        #ret returns true if a frame was read, false when if the video ended or had an error
        ret, frame = cap.read()

        if not ret:
            break  # End of video

        # save the frame every # frames
        if frame_count % frame_interval == 0:
            # Save the frame
            name, ext = os.path.splitext(filename)
            image_filename = os.path.join(output_folder, f"{name}_frame_{saved_frames_count:04d}.jpg")
            cv2.imwrite(image_filename, frame)
            saved_frames_count += 1

        frame_count += 1

    cap.release()
    print(f"Extracted frames every {interval_seconds} seconds")
    print(f"Extraction complete. {saved_frames_count} frames saved to {output_folder}")

#returns a lower and upper value 
def make_tolerances(min, max, tolerance, value):
    outL = 0
    outU = 1

    if min > max:
        t1 = max
        max = min
        min = t1
    
    # set the upper
    if value + tolerance > max:
        outU = max
    else:
        outU = value + tolerance
    
    # set the lower
    if value - tolerance < 0:
        outL = 0
    else:
        outL = value - tolerance

    #check if upper > lower
    if outL > outU:
        t1 = outU
        outU = outL
        outL = t1

    return outL, outU

# Define tolerance
# hue = color
# sat = intensity
#value = lightness/darkness
hue_tol = 5
sat_tol = 50
val_tol = 40

# hue_tol = 1
# sat_tol = 1
# val_tol = 1

# define getters + setters to change the tolerances
def get_hue_tol():
    return hue_tol
def set_hue_tol(tol):
    if int(tol) > 0:
        hue_tol = int(tol)

def get_sat_tol():
    return sat_tol
def set_sat_tol(tol):
    if int(tol) > 0:
        sat_tol = int(tol)

def get_val_tol():
    return val_tol
def set_val_tol(tol):
    if int(tol) > 0:
        val_tol = int(tol)

In [ ]:
# use this code block to extract all videos in folder
# ##Extract the frames
# frame_capture_rate = 0.4
# for filename in os.listdir(video_folder_path):
#     file_path = os.path.join(video_folder_path, filename)
#     print(filename)
#     full_file_path = video_folder_path + "\\" + filename
#     extract_frames_every_second(video_folder_path, filename, rawImage_folder_path, frame_capture_rate)

# ##Extract the frames
# frame_capture_rate = 0.4
# file_path = os.path.join(video_folder_path, "video01.mp4")
# full_file_path = video_folder_path + "\\" + "video01.mp4"
# extract_frames_every_second(video_folder_path, "video01.mp4", rawImage_folder_path, frame_capture_rate)

Extracted frames every 0.4 seconds
Extraction complete. 39 frames saved to .\BetaMove\rawImages


In [ ]:
#video01.mp4_frame_0000.jpg
hue_tol = 5
sat_tol = 50
val_tol = 40

title_string = ""

# used for detect_route input + output
detect_route_state = {
    "use_image": None,      # input: input image/mask
    "og_image": None,       # input: input image/mask
    "in_hsv": None,         # input: the input hsv 
    "hue_tol": hue_tol,     # input: hue tolerance 
    "sat_tol": sat_tol,     # input: saturation tolerance 
    "val_tol": val_tol,     # input: value tolerance 
    "cutout": None,      # output: the cutout of the route based on a selected pixel
    "mask": None,        # output: the mask of the route based on a selected pixel
    "p_hsv": None,       # output: the hsv of the selected pixel
    "x": None,           # output: the x of the pixel selected
    "y": None            # output: the y of the pixel selected
}

def prepare_raw_image(imageName, scale = 0.5):
    # Load image
    full_image_path = os.path.join(rawImage_folder_path, imageName)
    image = cv2.imread(full_image_path)

    # resize the image if needed
    image = cv2.resize(image, (0, 0), fx=scale, fy=scale)

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) # converts bgr to hsv(hue, sat, value) witch is needed

    # Clone for display
    display_image01 = image.copy()
    return image, display_image01

def prepare_image(imageOrMask, scale = 1):
    # resize the image if needed
    imageOrMask = cv2.resize(imageOrMask, (0, 0), fx=scale, fy=scale)

    hsv = cv2.cvtColor(imageOrMask, cv2.COLOR_BGR2HSV) # converts bgr to hsv(hue, sat, value) witch is needed

    # Clone for display
    display_image02 = imageOrMask.copy()
    return imageOrMask, display_image02, hsv

def remove_background(image, lower_bg, upper_bg):
    """
    Separates background (whitish-gray) from the foreground.

    Args:
        image (numpy.ndarray): Input BGR image.
        lower_bg (list or np.array): Lower HSV bound of background.
        upper_bg (list or np.array): Upper HSV bound of background.

    Returns:
        background_only (numpy.ndarray): Image showing only the background.
        foreground_only (numpy.ndarray): Image showing only the foreground.
        bg_mask (numpy.ndarray): Binary mask of background.
    """
    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Make background mask
    lower_bg = np.array(lower_bg, dtype=np.uint8)
    upper_bg = np.array(upper_bg, dtype=np.uint8)
    bg_mask = cv2.inRange(hsv, lower_bg, upper_bg)

    # Optional: clean up mask (remove noise, fill holes)
    kernel = np.ones((3, 3), np.uint8)
    bg_mask = cv2.morphologyEx(bg_mask, cv2.MORPH_OPEN, kernel)  # remove specks
    bg_mask = cv2.morphologyEx(bg_mask, cv2.MORPH_CLOSE, kernel) # close holes

    # Extract background & foreground
    background_only = cv2.bitwise_and(image, image, mask=bg_mask)
    fg_mask = cv2.bitwise_not(bg_mask)
    foreground_only = cv2.bitwise_and(image, image, mask=fg_mask)

    return background_only, foreground_only, bg_mask
mask_filename_global = ""
def detect_route(event, x, y, flags, param):
    title_string = ""
    if event == cv2.EVENT_LBUTTONDOWN:
        # Get HSV value of clicked pixel
        pixel_hsv = param["in_hsv"][y, x]
        param["x"] = x
        param["y"] = y
        print(f"pixel Values: {pixel_hsv}")

        #hue
        lower_hue, upper_hue = make_tolerances(0, 179, param["hue_tol"], pixel_hsv[0])
        #sat
        lower_sat, upper_sat = make_tolerances(0, 255, param["sat_tol"], pixel_hsv[1])
        #value
        lower_val, upper_val = make_tolerances(0, 255, param["val_tol"], pixel_hsv[2])

        lower = np.array([lower_hue,lower_sat,lower_val], dtype=np.uint8) # cast to make sure datatypes are the same
        upper = np.array([upper_hue,upper_sat,upper_val], dtype=np.uint8) # cast to make sure datatypes are the same
        
        # Create mask
        mask = cv2.inRange(hsv, lower, upper)

        # de-noise the mask through opening + closing
        kernel = np.ones((3, 3), np.uint8) 
        # oppend_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        # closed_mask = cv2.morphologyEx(oppend_mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        # #  to fill any holes in the holds
        # kernel = np.ones((5, 5), np.uint8)
        # mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # create a cutout (mask + og picture)
        cutout = cv2.bitwise_and(param["use_image"], param["use_image"], mask=mask)

        # Show updates
        mask_filename = os.path.join(rawImage_folder_path, f"holdsMask.jpg")
        mask_filename_global = mask_filename
        cv2.imwrite(mask_filename, mask)
        cv2.imshow("Mask", mask)
        cv2.imshow("Result", cutout)

        print(f"Clicked HSV: {pixel_hsv}, Lower: {lower}, Upper: {upper}")
        param["cutout"] = cutout
        param["mask"] = mask
        param["p_hsv"] = pixel_hsv
        #return cutout, mask

        # draw the boundries on the holds
        group_holds(param)

def group_holds(param, min_area = 25, max_dist = 150):
            # Parameters
        min_area = 25
        max_dist = 150

        # Find contours (white blobs in mask)
        contours, _ = cv2.findContours(param["mask"], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        output = param["og_image"].copy()
        centers = []

        # Step 1: detect blobs(holds) and get centers
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > min_area:
                (x, y), radius = cv2.minEnclosingCircle(cnt)
                center = (int(x), int(y))
                centers.append(center)

                cv2.circle(output, center, int(radius), (0, 255, 0), 2)
                cv2.putText(output, f"{int(area)}", (center[0] - 20, center[1] - 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Step 2: build a single chain starting from a chosen circle (ex. first circle)
        used = set()
        chain = []
        once = True

        if centers:
            # Find closest center to selected pixel (y = 1000 to start from the bottom)
            distances = [(math.dist((param["x"], 1000), center), idx) for idx, center in enumerate(centers)]
            distances.sort(key=lambda x: x[0])
            closest_distance, closest_idx = distances[0]

            current_idx = closest_idx            # starting circle (c1)
            used.add(current_idx)
            chain.append(current_idx)

            while True:
                c1 = centers[current_idx]
                # find closest unused neighbor
                distances = [(math.dist(c1, c2), j) for j, c2 in enumerate(centers) if j not in used]
                if not distances:
                    break  # no unused neighbors left
                distances.sort(key=lambda x: x[0])
                dist, next_idx = distances[0]
                if dist > max_dist:
                    break  # next neighbor too far

                # draw line
                c2 = centers[next_idx]
                #cv2.line(output, c1, c2, (255, 0, 0), 2)
                cv2.arrowedLine(output, c1, c2, (255, 0, 0), 2, tipLength=0.2)

                # move to next circle
                used.add(next_idx)
                chain.append(next_idx)
                current_idx = next_idx

                cv2.imshow("Chained Connections", output)
                image_filename = os.path.join(rawImage_folder_path, f"outPutWtConnections.jpg")
                cv2.imwrite(image_filename, output)

lower_bg = [0, 0, 104]
upper_bg = [178, 33, 244]

useImage, display_image01 = prepare_raw_image("video01_frame_0000.jpg")
# useImage, display_image01 = prepare_raw_image("video01_3_frame_0000.jpg")
background_only, foreground_only, bg_mask = remove_background(useImage, lower_bg, upper_bg)

lower_bg = [96, 138, 124]
upper_bg = [106, 238, 204]

background_only, foreground_only, bg_mask = remove_background(foreground_only, lower_bg, upper_bg)

useImage, display_image02, hsv = prepare_image(foreground_only)

image_filename = os.path.join(rawImage_folder_path, f"outPuForground.jpg")
cv2.imwrite(image_filename, useImage)

detect_route_state["use_image"] = useImage
detect_route_state["og_image"] = display_image01
detect_route_state["in_hsv"] = hsv


cv2.imshow(f"Image", display_image02)

cv2.setMouseCallback(f"Image", detect_route, detect_route_state)


#group_holds(detect_route_state["mask"])

# Show original image
# cv2.imshow(f"Image", display_image)
# cv2.setMouseCallback(f"Image", mouse_callback)

print("Click on the image to pick a color (press 'q' to quit).")

def update_title():
    t1 = {detect_route_state["p_hsv"]}
    cv2.setWindowTitle("Image", f"Image - Mode: {title_string} : HVS {hue_tol},{val_tol},{sat_tol}, Pixel value = {t1}")

update_title()  # set initial title

# Start in hue mode by default
mode = "h"   # can be "h", "s", or "v"

while True:
    key = cv2.waitKey(1) & 0xFF
    

    if key == ord('q'):   # Quit
        break

    # # --- Switch modes ---
    # elif key == ord('h'):
    #     mode = "h"
    #     print("Mode: Hue tolerance")
    #     title_string = "Hue"
    #     update_title()
    # elif key == ord('v'):
    #     mode = "v"
    #     print("Mode: Value tolerance")
    #     title_string = "Value"
    #     update_title()
    # elif key == ord('s'):
    #     mode = "s"
    #     print("Mode: Saturation tolerance")
    #     title_string = "Sat"
    #     update_title()

    # # --- Adjust based on mode ---
    # elif key == ord('='):   # '+' key
    #     if mode == "h":
    #         hue_tol = min(hue_tol + 1, 179)
    #         print(f"Hue tolerance increased to {hue_tol}")
    #     elif mode == "s":
    #         sat_tol = min(sat_tol + 1, 255)
    #         print(f"Saturation tolerance increased to {sat_tol}")
    #     elif mode == "v":
    #         val_tol = min(val_tol + 1, 255)
    #         print(f"Value tolerance increased to {val_tol}")
    #     update_title()

    # elif key == ord('-'):   # '-' key
    #     if mode == "h":
    #         hue_tol = max(hue_tol - 1, 0)
    #         print(f"Hue tolerance decreased to {hue_tol}")
    #     elif mode == "s":
    #         sat_tol = max(sat_tol - 1, 0)
    #         print(f"Saturation tolerance decreased to {sat_tol}")
    #     elif mode == "v":
    #         val_tol = max(val_tol - 1, 0)
    #         print(f"Value tolerance decreased to {val_tol}")
    #     update_title()


cv2.destroyAllWindows() # kill the popout windows

Click on the image to pick a color (press 'q' to quit).
pixel Values: [174 175 131]
Clicked HSV: [174 175 131], Lower: [169 125  91], Upper: [179 225 171]
